In [2]:
import numpy as np
from tqdm import tqdm 
import torch
import numpy as np
import random
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import sys
import os
os.chdir('d:\\1st\\BPs_BPTK')
sys.path.append("Python") 
from BPS_init_function_MultiParas import BPS_BPTK_MultiParas
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False # 用来正常显示负号
from sklearn.metrics import r2_score
import scipy.stats as stats
import os
os.chdir('d:\\1st\\BPs_BPTK')

ModuleNotFoundError: No module named 'BPS_init_function_MultiParas'

In [ ]:
X =  np.load("Python\optim\Time_DualNN14\database\input_add.npy")  
y = np.load("Python\optim\Time_DualNN14\database\label_add.npy")  